In [1]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)

def modified_gram_schmidt(A):
    """
    Implements the Modified Gram-Schmidt orthogonalization to get the QR decomposition of matrix A.
    A = QR
    """
    A = A.astype(float)  # Ensure A is of float type
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for i in range(n-1):
        R[i, i] = np.linalg.norm(A[:, i])
        Q[:, i] = A[:, i] / R[i, i]
        R[i, i+1:n] = np.dot(Q[:, i].T, A[:, i+1:n])
        A[:, i+1:n] -= np.outer(Q[:, i], R[i, i+1:n])
    R[n-1, n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1] / R[n-1, n-1]
    return Q, R

# Householder reflection
def householder_reflection(a):
    v = a.copy()
    v[0] = v[0] + np.copysign(np.linalg.norm(v), a[0])
    v = v / np.linalg.norm(v)
    return np.eye(len(a)) - 2 * np.outer(v, v)

# QR decomposition using Householder reflections
def qr_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for i in range(min(m, n)):
        H = np.eye(m)
        H[i:, i:] = householder_reflection(R[i:, i])
        Q = Q @ H
        R = H @ R
    return Q, R

#dynamic size QR decomposition
def QRDecomposition():
    A = tf.input([-1, -1], tf.float32)

    m, n = A.shape
    Q = tf.zeros([m, n])
    R = tf.zeros([n, n])
    j = tf.index(0, [m])

    def loop_body(i):
        R[i, i] = tf.norm(A[j, i])
        Q[j, i] = A[j, i] / R[i, i]

        p, k = tf.index_grid([0, i + 1], [m, n])
        t, = tf.index_grid([i+1], [n])
        R[i, t] = tf.sum(Q[p, i] * A[p, k], axis=0)
        A[p, k] -= Q[p, i] * R[i, k]

    tf.loop(loop_body, 0, n-1, 1)

    R[n-1, n-1] = tf.norm(A[j, n-1])
    Q[j, n-1] = A[j, n-1] / R[n-1, n-1]

    return [Q, R]

qr = tf.compile(QRDecomposition)


TensorFrost module loaded!
QRDecomposition:
  Kernel count: 8
  Intermediate buffers: 0
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 394
  IR Compile time: 1.796400 ms
  Compiler time: 1523.813477 ms



In [2]:
#generate random matrix
A = np.random.rand(5, 5)

#compute QR decomposition
Q, R = modified_gram_schmidt(A)

#compute QR decomposition using TensorFrost
Atf = tf.tensor(A)
Qtf, Rtf = qr(Atf)
Qnp = Qtf.numpy
Rnp = Rtf.numpy

#check if QR decomposition is correct
print("QR decomposition is correct:", np.allclose(A, np.dot(Q, R)))
print("QR decomposition using TensorFrost is correct:", np.allclose(A, np.dot(Qnp, Rnp)))

#check error
print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qnp, Rnp)))

#print Q and R
print("Q:\n", Qnp)
print("R:\n", Rnp)


QR decomposition is correct: True
QR decomposition using TensorFrost is correct: True
Error: 1.1102230246251565e-16
Error using TensorFrost: 9.80524335107431e-08
Q:
 [[ 0.12411568  0.7326195  -0.22203594  0.49314734  0.39416975]
 [ 0.57611185  0.37130576 -0.2244676  -0.6756834  -0.1526212 ]
 [ 0.17443486  0.29325262  0.45711353  0.3281024  -0.75297487]
 [ 0.36590293 -0.04185599  0.77686805 -0.08293105  0.50394624]
 [ 0.6988403  -0.4874952  -0.29637414  0.4309628  -0.02009838]]
R:
 [[ 1.0132961   0.9248877   0.8464499   1.1308571   0.9522835 ]
 [ 0.          0.9544738   0.7976799   0.76563853  0.13753597]
 [ 0.          0.          0.6116486   0.28257155 -0.01758742]
 [ 0.          0.          0.          0.6672542   0.3415734 ]
 [ 0.          0.          0.          0.          0.28049704]]


In [3]:
#performance test
import time
A = np.random.rand(5000, 5000).astype(np.float32)

#naive NumPy QR decomposition
#start = time.time()
#Q, R = modified_gram_schmidt(A)
#print("Time for naive NumPy QR decomposition:", time.time() - start)

#TensorFrost QR decomposition
Atf = tf.tensor(A)
start = time.time()
Qtf, Rtf = qr(Atf)
print("Time for TensorFrost QR decomposition:", time.time() - start)

#householder QR decomposition
#start = time.time()
#Q, R = qr_decomposition(A)
#print("Time for householder QR decomposition:", time.time() - start)

#built-in NumPy QR decomposition
start = time.time()
Q, R = np.linalg.qr(A)
print("Time for built-in NumPy QR decomposition:", time.time() - start)

print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qtf.numpy, Rtf.numpy)))

Time for TensorFrost QR decomposition: 0.18627309799194336
Time for built-in NumPy QR decomposition: 4.043566703796387
Error: 0.0012291038
Error using TensorFrost: 0.001878134
